In [26]:
import pandas as pd
from pathlib import Path

In [27]:
ROOT = Path.cwd().resolve().parent
DATA = ROOT / "Data" 
GDP  = DATA / "gdp-per-capita-worldbank" / "gdp-per-capita-worldbank.csv"
LIFE = DATA / "happiness-cantril-ladder" / "happiness-cantril-ladder.csv"

In [28]:
df_gdp = pd.read_csv(GDP)
df_life = pd.read_csv(LIFE)

In [29]:
df_gdp.head()

,Entity,Code,Year,"GDP per capita, PPP (constant 2021 international $)",World region according to OWID
0,Afghanistan,AFG,2000,1617.8264,NaN
1,Afghanistan,AFG,2001,1454.1108,NaN
2,Afghanistan,AFG,2002,1774.3087,NaN
3,Afghanistan,AFG,2003,1815.9282,NaN
4,Afghanistan,AFG,2004,1776.9182,NaN


In [30]:
df_life.head()

,Entity,Code,Year,Cantril ladder score
0,Afghanistan,AFG,2011,4.258
1,Afghanistan,AFG,2012,4.040
2,Afghanistan,AFG,2014,3.575
3,Afghanistan,AFG,2015,3.360
4,Afghanistan,AFG,2016,3.794


In [31]:
df_gdp.isnull().sum()

Entity                                                    0
Code                                                    435
Year                                                      0
GDP per capita, PPP (constant 2021 international $)      75
World region according to OWID                         7039
dtype: int64

In [32]:
df_life.isnull().sum()

Entity                    0
Code                    130
Year                      0
Cantril ladder score      0
dtype: int64

In [33]:
len(df_life['Entity'].unique())

178

In [34]:
len(df_gdp['Entity'].unique())

285

In [35]:
285-178

107

Based on the information shown, we can conclude that the merge should be done using `Entity` rather than `Code`, because some entities do not have a code in the dataset. If we merged by `code`, we would lose information.

Additionally, the GDP dataset contains more entities—107 more, to be precise. Therefore, merging by code would result in losing at least 107 countries in the final dataset.

In [36]:
df_gdp.columns

Index(['Entity', 'Code', 'Year',
       'GDP per capita, PPP (constant 2021 international $)',
       'World region according to OWID'],
      dtype='object')

In [37]:
df_life = df_life[['Entity', 'Year', 'Code', 'Cantril ladder score']]
df_gdp = df_gdp[['Entity', 'Year', 'GDP per capita, PPP (constant 2021 international $)']]

In [38]:
df_life_gdp = pd.merge(left=df_life, right=df_gdp ,on=['Entity','Year'], how='inner')

In [39]:
df_life_gdp

,Entity,Year,Code,Cantril ladder score,"GDP per capita, PPP (constant 2021 international $)"
0,Afghanistan,2011,AFG,4.258,2757.0525
1,Afghanistan,2012,AFG,4.040,2985.3190
2,Afghanistan,2014,AFG,3.575,3017.9426
3,Afghanistan,2015,AFG,3.360,2967.6921
4,Afghanistan,2016,AFG,3.794,2958.7854
...,...,...,...,...,...
1970,Zimbabwe,2020,ZWE,3.145,2987.2683
1971,Zimbabwe,2021,ZWE,2.995,3184.7847
1972,Zimbabwe,2022,ZWE,3.204,3323.1184
1973,Zimbabwe,2023,ZWE,3.341,3442.2488


In [40]:
len(df_life_gdp['Entity'].unique())

168

We can see that we have 168 entities because the merge also takes the year into account, and it is possible that some countries did not match for certain years.

In [41]:
Mex = df_life_gdp[df_life_gdp['Entity']=='Mexico']
Mex

,Entity,Year,Code,Cantril ladder score,"GDP per capita, PPP (constant 2021 international $)"
1173,Mexico,2011,MEX,6.740,20457.600
1174,Mexico,2012,MEX,7.088,20898.936
1175,Mexico,2014,MEX,7.187,21069.766
1176,Mexico,2015,MEX,6.778,21408.928
1177,Mexico,2016,MEX,6.578,21578.264
1178,Mexico,2017,MEX,6.488,21777.523
1179,Mexico,2018,MEX,6.595,21997.773
1180,Mexico,2019,MEX,6.465,21704.186
1181,Mexico,2020,MEX,6.317,19728.482
1182,Mexico,2021,MEX,6.128,20782.588


In [42]:
df_life_gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975 entries, 0 to 1974
Data columns (total 5 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               1975 non-null   object 
 1   Year                                                 1975 non-null   int64  
 2   Code                                                 1923 non-null   object 
 3   Cantril ladder score                                 1975 non-null   float64
 4   GDP per capita, PPP (constant 2021 international $)  1972 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 77.3+ KB


Now we have to make the datasets of the entities for 2021, 2022, 2023, 2024 the amount of entities may change in the datasets, as we can see in Mexico there is no record for 2013. That may be the case for some entities between 2021 and 2024.

In [43]:
DATA_YEAR = DATA / "data_for_year"

In [ ]:
years = [2021, 2022, 2023, 2024]

for year in years:
    (
        df_life_gdp[df_life_gdp["Year"] == year]
        .to_csv(DATA_YEAR / f"lifesat_gdp_{year}.csv", index=False)
    )